In [5]:
from janome.tokenizer import Tokenizer

# 開始、終端記号を設定する。
BEGIN = '__BEGIN__'
END = '__END__'
# 実際に一文を3単語ずつに分けてみる。
sentence = 'おいしいビールを飲もう'

t = Tokenizer()
# ここで型変換しないとgeneratorのリストができてしまう
# 分かち書きした文章に開始終端記号を追加してリストにする
words = list(t.tokenize(sentence, wakati=True))
words = [BEGIN] + words + [END]

# ['__BEGIN__', 'おいしい', 'ビール', 'を', '飲も', 'う', '__END__']
# 3単語ずつを1つのリストにすると全体の単語数ー２の回数回さないといけない
three_words_list = []
for i in range(len(words) - 2):
    #3単語ずつ一つリストに追加していく
    three_words_list.append(words[i:i + 3])
three_words_list

[['__BEGIN__', 'おいしい', 'ビール'],
 ['おいしい', 'ビール', 'を'],
 ['ビール', 'を', '飲も'],
 ['を', '飲も', 'う'],
 ['飲も', 'う', '__END__']]

In [6]:
from collections import Counter
# さっきのをモジュール化する
def get_three_words_list(sentence):
    """文章を3単語の組にして返す"""
    t = Tokenizer()
    words = list(t.tokenize(sentence, wakati=True))
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list

sentences = ['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生','ビールを飲んで休もう', 'おいしいお酒が飲みたい']
three_words_list = []
# リストに追加されている3単語つづりの内容の出現率をカウントしてリストに追加
for sentence in sentences:
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
three_words_count


Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('おいしい', 'ビール', 'を'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('を', '飲も', 'う'): 2,
         ('飲も', 'う', '__END__'): 2,
         ('__BEGIN__', 'ビール', 'を'): 2,
         ('おいしい', 'ビール', 'は'): 1,
         ('ビール', 'は', '生'): 1,
         ('は', '生', '__END__'): 1,
         ('ビール', 'を', '飲ん'): 1,
         ('を', '飲ん', 'で'): 1,
         ('飲ん', 'で', '休も'): 1,
         ('で', '休も', 'う'): 1,
         ('休も', 'う', '__END__'): 1,
         ('__BEGIN__', 'おいしい', 'お'): 1,
         ('おいしい', 'お', '酒'): 1,
         ('お', '酒', 'が'): 1,
         ('酒', 'が', '飲み'): 1,
         ('が', '飲み', 'たい'): 1,
         ('飲み', 'たい', '__END__'): 1})

In [7]:
def generate_marcov_dict(three_words_count):
    """マルコフ連鎖での文章作成用の辞書データを作成する"""
    # key : 前半二つの単語
    # value : 次の単語とその重みとして出現回数を追加する
    markov_dict = {}
    #　前半の3単語とその重みで分ける
    #three_words = 3単語, count = 重み
    # for文の処理の中でさらに前半2単語後半1単語に分ける。
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]
        next_word = three_words[2]
        # keyが新出の場合のみ辞書に追加
        if two_words not in markov_dict:
            markov_dict[two_words] = {'words': [], 'weights': []}
        # keyが既出でvalueが新出のときはここで追加されていく。
        markov_dict[two_words]['words'].append(next_word)
        markov_dict[two_words]['weights'].append(count)
    return markov_dict

markov_dict = generate_marcov_dict(three_words_count)
markov_dict


{('__BEGIN__', 'おいしい'): {'words': ['ビール', 'お'], 'weights': [2, 1]},
 ('おいしい', 'ビール'): {'words': ['を', 'は'], 'weights': [1, 1]},
 ('ビール', 'を'): {'words': ['飲も', '飲ん'], 'weights': [2, 1]},
 ('を', '飲も'): {'words': ['う'], 'weights': [2]},
 ('飲も', 'う'): {'words': ['__END__'], 'weights': [2]},
 ('__BEGIN__', 'ビール'): {'words': ['を'], 'weights': [2]},
 ('ビール', 'は'): {'words': ['生'], 'weights': [1]},
 ('は', '生'): {'words': ['__END__'], 'weights': [1]},
 ('を', '飲ん'): {'words': ['で'], 'weights': [1]},
 ('飲ん', 'で'): {'words': ['休も'], 'weights': [1]},
 ('で', '休も'): {'words': ['う'], 'weights': [1]},
 ('休も', 'う'): {'words': ['__END__'], 'weights': [1]},
 ('おいしい', 'お'): {'words': ['酒'], 'weights': [1]},
 ('お', '酒'): {'words': ['が'], 'weights': [1]},
 ('酒', 'が'): {'words': ['飲み'], 'weights': [1]},
 ('が', '飲み'): {'words': ['たい'], 'weights': [1]},
 ('飲み', 'たい'): {'words': ['__END__'], 'weights': [1]}}

In [8]:
# ここまでの内容をcollectionライブラリの関数defaultdictを使って簡略化する。
from collections import defaultdict

def get_first_word_and_count(three_words_count):
    """最初の単語を選択するための辞書データを作成する"""
    # 引数をintとすると初期値は0になる
    first_word_count = defaultdict(int)
# 3単語ずつの辞書を回してBEGINから始まる単語を検索する。またその回数をカウントする。
# key : 初めの単語
# value : keyの出現回数
    for three_words, count in three_words_count.items():
        if three_words[0] == BEGIN:
            next_word = three_words[1]
            first_word_count[next_word] += count

    return first_word_count

get_first_word_and_count(three_words_count)


defaultdict(int, {'おいしい': 3, 'ビール': 2})

In [9]:
def get_first_words_weights(three_words_count):
    """最初の単語と重みのリストを作成する"""
    # ひとつ前で得た辞書データの形式を変換する。
    # random.choices()に渡しやすくするためにどっちもリストにして保管
    first_word_count = get_first_word_and_count(three_words_count)
    words = []
    weights = []
    for word, count in first_word_count.items():
        words.append(word)
        weights.append(count)

    return words, weights

first_words, first_weights = get_first_words_weights(three_words_count)
first_words, first_weights

(['おいしい', 'ビール'], [3, 2])

In [10]:
import random

def generate_text(first_words, first_weights, markov_dict):
    """入力された辞書データをもとに文章を生成する"""

    # 最初の単語を選ぶ
    first_word = random.choices(first_words, weights=first_weights)[0]
    generate_words = [BEGIN, first_word]
    while True:
        pair = tuple(generate_words[-2:])
        words = markov_dict[pair]['words']
        weights = markov_dict[pair]['weights']
        next_word = random.choices(words, weights=weights)[0]
        if next_word == END:
            break
        generate_words.append(next_word)

    return ''.join(generate_words[1:])

for i in range(5):
    text = generate_text(first_words, first_weights, markov_dict)
    print(text)



ビールを飲もう
おいしいビールは生
ビールを飲もう
おいしいビールは生
おいしいビールを飲もう


In [11]:
# ここから実データを取得して実際に文章を生成する。
import requests

#人間失格のURL
url = 'https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip'
r = requests.get(url)
content = r.content


In [12]:
# ioは標準ライブラリ
# io.BytesIO()を使ってバイナリデータをzipとして扱えるようにする
import io
import zipfile

# バイナリをファイルのように変換
f = io.BytesIO(content)
# zipファイルを開く
zipf = zipfile.ZipFile(f)
#開いたファイルの一覧を取得
namelist = zipf.namelist()
namelist

['ningen_shikkaku.txt']

In [13]:
data = zipf.read(namelist[0])
original_text = data.decode('Shift_JIS')
print(original_text[:500])

人間失格
太宰治

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）従姉妹《いとこ》

｜：ルビの付く文字列の始まりを特定する記号
（例）昔｜気質《かたぎ》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
（例）［＃３字下げ］はしがき［＃「はしがき」は大見出し］
-------------------------------------------------------

［＃３字下げ］はしがき［＃「はしがき」は大見出し］


　私は、その男の写真を三葉、見たことがある。
　一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？　けれども、鈍い人たち（


In [ ]:
import re

first_sentence = '私は、その男の写真を三葉、見たことがある'
last_sentence = '神様みたいないい子でした'
_, text = original_text.split(first_sentence)
text, _ = text.split(last_sentence)
text = first_sentence + text + last_sentence

text = text.replace('|', '').replace(' ', '')
text = re.sub(' [')